In [ ]:
!pip install ortools

In [8]:
# pip install ortools
from ortools.sat.python import cp_model
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple


@dataclass
class Template:
    """A single shift template with a binary coverage vector over the open hours."""
    name: str                 # e.g., "S06-14" or "S06-14_L4" (L= lunch offset)
    cover: List[int]          # length = hours_open; 1 if this shift covers that hour, else 0


def build_basic_templates(
    hours_open: int = 16,
    shift_length: int = 8,
    allowed_starts: Optional[List[int]] = None,
) -> List[Template]:
    """
    Build simple 8-hour (default) templates with no break:
    - hours indexed 0..hours_open-1 (e.g., 0 == opening hour, 15 == last open hour)
    - a template that starts at s covers hours [s, s+shift_length-1]
    """
    if allowed_starts is None:
        allowed_starts = list(range(0, hours_open - shift_length + 1))

    templates: List[Template] = []
    for s in allowed_starts:
        cover = [0] * hours_open
        for h in range(s, s + shift_length):
            cover[h] = 1
        name = f"S{str(s).zfill(2)}-{str(s+shift_length-1).zfill(2)}"
        templates.append(Template(name=name, cover=cover))
    return templates

def optimize_daily_heads(
    hourly_requirement: List[int],
    templates: List[Template],
    *,
    max_total_heads: Optional[int] = None,
    solver_time_limit_s: Optional[int] = 10,
) -> Dict:
    """
    Solve: minimize sum(x_t) s.t. for each hour h, sum_t cover[t,h] * x_t >= R[h].
    x_t are nonnegative integers (number of agents scheduled on template t).
    """
    H = len(hourly_requirement)
    T = len(templates)

    model = cp_model.CpModel()
    x = [model.NewIntVar(0, 10_000, f"x_{t.name}") for t in templates]

    # Hourly coverage constraints
    for h in range(H):
        model.Add(
            sum(x[t] * templates[t].cover[h] for t in range(T)) >= hourly_requirement[h]
        )

    # Optional cap
    total_heads = model.NewIntVar(0, 10_000, "total_heads")
    model.Add(total_heads == sum(x))
    if max_total_heads is not None:
        model.Add(total_heads <= max_total_heads)

    # Objective
    model.Minimize(total_heads)

    # Solve
    solver = cp_model.CpSolver()
    if solver_time_limit_s is not None:
        solver.parameters.max_time_in_seconds = float(solver_time_limit_s)
    solver.parameters.num_search_workers = 8  # parallel search

    status = solver.Solve(model)

    result = {
        "status": solver.StatusName(status),
        "objective_value": None,
        "total_heads": None,
        "by_template": [],
        "coverage_by_hour": None,
        "requirement": hourly_requirement,
    }

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        by_template = []
        for t in range(T):
            ct = solver.Value(x[t])
            if ct > 0:
                by_template.append((templates[t].name, ct))
        # Compute realized coverage
        cov = [0] * H
        for h in range(H):
            cov[h] = sum(solver.Value(x[t]) * templates[t].cover[h] for t in range(T))
        result.update(
            {
                "objective_value": solver.ObjectiveValue(),
                "total_heads": int(solver.Value(total_heads)),
                "by_template": sorted(by_template, key=lambda z: z[0]),
                "coverage_by_hour": cov,
            }
        )
    return result


def pretty_print_solution(sol: Dict):
    print(f"Status         : {sol['status']}")
    if sol["total_heads"] is None:
        return
    print(f"Total heads    : {sol['total_heads']}")
    print("\nShift mix (template -> count):")
    for name, ct in sol["by_template"]:
        print(f"  {name:>12} : {ct}")
    print("\nHour | Required | Covered | Slack")
    print("-" * 38)
    for h, (r, c) in enumerate(zip(sol["requirement"], sol["coverage_by_hour"])):
        print(f"{h:>4} | {r:>8} | {c:>7} | {c - r:>5}")


if __name__ == "__main__":
    # === Example usage ===
    # 16-hour day requirement (already includes shrinkage).
    # Replace this list with your actual hourly staffing requirement.
    R = [6, 7, 8, 10, 12, 14, 16, 18, 18, 17, 15, 12, 10, 8, 7, 6]

    HOURS_OPEN = 16
    SHIFT_LEN = 8

    # Option A: No-break templates, starts every hour
    base = build_basic_templates(hours_open=HOURS_OPEN, shift_length=SHIFT_LEN)

    # Option B (recommended if lunches are NOT baked into R):
    # Create lunch variants at offsets 3/4/5 within each shift (1-hour lunch).
    # lunch_variants = add_one_hour_lunch_variants(base, lunch_offsets_within_shift=[3, 4, 5])
    # You can choose either `base` OR `lunch_variants` OR both; using both lets the solver
    # decide which to pick. Here we include both:
    templates = base

    sol = optimize_daily_heads(R, templates, solver_time_limit_s=5)
    pretty_print_solution(sol)


Status         : OPTIMAL
Total heads    : 24

Shift mix (template -> count):
        S00-07 : 6
        S01-08 : 1
        S02-09 : 1
        S03-10 : 2
        S04-11 : 4
        S05-12 : 2
        S06-13 : 1
        S07-14 : 1
        S08-15 : 6

Hour | Required | Covered | Slack
--------------------------------------
   0 |        6 |       6 |     0
   1 |        7 |       7 |     0
   2 |        8 |       8 |     0
   3 |       10 |      10 |     0
   4 |       12 |      14 |     2
   5 |       14 |      16 |     2
   6 |       16 |      17 |     1
   7 |       18 |      18 |     0
   8 |       18 |      18 |     0
   9 |       17 |      17 |     0
  10 |       15 |      16 |     1
  11 |       12 |      14 |     2
  12 |       10 |      10 |     0
  13 |        8 |       8 |     0
  14 |        7 |       7 |     0
  15 |        6 |       6 |     0
